In [3]:
from samgeo import SamGeo2
import numpy as np
import os
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

In [ ]:
image = "tiff_testing/tile_16384_38912.tif"
sam2 = SamGeo2(
    model_id="sam2-hiera-large",
    apply_postprocessing=False,
    points_per_side=32,
    points_per_batch=64,
    pred_iou_thresh=0.6,
    stability_score_thresh=0.85,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.9,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=25.0,
    use_m2m=True,
)
sam2.generate(image)
sam2.save_masks(output="masks.tif")
sam2.show_anns(axis="off", alpha=0.7, output="annotations.tif")

In [ ]:
import rasterio
import numpy as np
from rasterio.features import shapes, rasterize
from shapely.geometry import shape
import matplotlib.pyplot as plt

# --- Paths ---
mask_path = "masks.tif"
clean_mask_path = "masks_clean.tif"

# --- Read mask ---
with rasterio.open(mask_path) as src:
    mask_data = src.read()  # all layers
    profile = src.profile
    transform = src.transform

# --- Combine all layers into a single 2D mask ---
mask_combined = np.max(mask_data, axis=0) > 0
mask_combined = mask_combined.astype(np.uint8)

# --- Extract polygons from combined mask ---
polygons = [shape(geom) for geom, val in shapes(mask_combined, mask=mask_combined, transform=transform)]
print(f"Total polygons detected: {len(polygons)}")

# --- Rasterize polygons back to a cleaned mask ---
clean_mask = rasterize([(poly, 1) for poly in polygons],
                       out_shape=mask_combined.shape,
                       transform=transform,
                       all_touched=True)

# --- Save cleaned mask ---
with rasterio.open(clean_mask_path, "w", **profile) as dst:
    dst.write(clean_mask, 1)

print(f"Cleaned mask saved at: {clean_mask_path}")

# --- Visualize original combined mask vs cleaned mask ---
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

axs[0].imshow(mask_combined, cmap="gray")
axs[0].set_title("Original Combined Mask")
axs[0].axis("off")

axs[1].imshow(clean_mask, cmap="gray")
axs[1].set_title("Cleaned Mask (Rasterized Polygons)")
axs[1].axis("off")

plt.show()


In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
import numpy as np

mask_path = "masks_clean.tif"
output_shapefile = "mask_boundaries.shp"

# Open the mask (first layer, or combine layers)
with rasterio.open(mask_path) as src:
    mask_data = src.read(1)  # first mask layer
    transform = src.transform
    crs = src.crs

# mask_data = np.max(src.read(), axis=0)

# shapes() extracts polygons from non-zero mask regions (here is the problem it dosent counts the connected neighbour polygon and sees all as one)
mask_polygons = (
    {"properties": {"value": int(v)}, "geometry": s}
    for s, v in shapes(mask_data, mask=mask_data>0, transform=transform)
)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(list(mask_polygons))
gdf.crs = crs  # preserve original CRS
print(f"Number of polygons detected: {len(gdf)}")
gdf.to_file(output_shapefile, driver="ESRI Shapefile")
print(f"Shapefile saved at: {output_shapefile}")


In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show

original_image_path = "tiff_testing/tile_16384_38912.tif"

with rasterio.open(original_image_path) as src:
    fig, ax = plt.subplots(figsize=(10,10))
    show(src, ax=ax)
    gdf.boundary.plot(ax=ax, edgecolor='red', linewidth=1)
    plt.title("Mask Boundaries Overlay")
    plt.axis("off")
    plt.show()


In [ ]:
import os
import rasterio
import numpy as np
from rasterio.features import shapes, rasterize
from shapely.geometry import shape
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show
from samgeo import SamGeo2

sam2 = SamGeo2(
    model_id="sam2-hiera-large",
    apply_postprocessing=False,
    points_per_side=32,
    points_per_batch=64,
    pred_iou_thresh=0.6,
    stability_score_thresh=0.85,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.9,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=25.0,
    use_m2m=True,
)


def process_and_visualize(image_path, output_dir="outputs"):
    """
    End-to-end processing of one image: SAM mask, cleaning, polygon extraction, shapefile, visualization
    """
    os.makedirs(output_dir, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    mask_path = os.path.join(output_dir, f"{base_name}_masks.tif")
    clean_mask_path = os.path.join(output_dir, f"{base_name}_masks_clean.tif")
    shapefile_path = os.path.join(output_dir, f"{base_name}_mask_boundaries.shp")
    annotation_path = os.path.join(output_dir, f"{base_name}_annotations.tif")

    sam2.generate(image_path)
    sam2.save_masks(output=mask_path)
    sam2.show_anns(axis="off", alpha=0.7, output=annotation_path)


    with rasterio.open(mask_path) as src:
        mask_data = src.read()
        profile = src.profile
        transform = src.transform

    mask_combined = np.max(mask_data, axis=0) > 0
    mask_combined = mask_combined.astype(np.uint8)

    polygons = [shape(geom) for geom, val in shapes(mask_combined, mask=mask_combined, transform=transform)]

    clean_mask = rasterize([(poly, 1) for poly in polygons],
                           out_shape=mask_combined.shape,
                           transform=transform,
                           all_touched=True)
    clean_mask = clean_mask.astype("uint8")

    with rasterio.open(clean_mask_path, "w", **profile) as dst:
        dst.write(clean_mask, 1)

    mask_polygons = (
        {"properties": {"value": int(v)}, "geometry": s}
        for s, v in shapes(clean_mask, mask=clean_mask > 0, transform=transform)
    )
    gdf = gpd.GeoDataFrame.from_features(list(mask_polygons))
    gdf.crs = profile.get('crs')
    gdf.to_file(shapefile_path, driver="ESRI Shapefile")

    with rasterio.open(image_path) as src:
        fig, axs = plt.subplots(1, 2, figsize=(16, 8))
        axs[0].imshow(mask_combined, cmap="gray")
        axs[0].axis("off")

        show(src, ax=axs[1])
        gdf.boundary.plot(ax=axs[1], edgecolor='red', linewidth=1)
        axs[1].axis("off")
        plt.tight_layout()
        plt.show()

    return {
        "image_path": image_path,
        "mask_path": mask_path,
        "clean_mask_path": clean_mask_path,
        "shapefile_path": shapefile_path,
        "num_polygons": len(polygons)
    }


# ---- main loop ----
image_folder = "tiff_testing"
results = []

for file in sorted(os.listdir(image_folder)):
    if file.lower().endswith(".tif"):
        full_path = os.path.join(image_folder, file)
        print(f"Processing {full_path} ...")
        res = process_and_visualize(full_path)
        results.append(res)

print("All images processed.")
